Объединяю и чищу объявления WW2

In [64]:
# импорт библиотек
import pandas as pd
pd.options.mode.chained_assignment = None  # отключаю предупреждения default='warn'

import os

# прогресс-бар
from tqdm.notebook import trange, tqdm

### Объединяю файлы

In [53]:
# путь к файлам

file_path = 'C:/00_Data/temp_data/'

In [54]:
# вытаскиваю названия файлов в список file_list

file_list = []

for root, dirs, files in os.walk(file_path + 'original/'):
    
    for filename in files:
        file_list += [filename]

In [55]:
# объединяю в один датафрейм

new_work = pd.DataFrame() # новый датафрейм

for file in tqdm(file_list):
    temp = pd.read_excel(file_path + 'original/' + file)
    
    temp.drop_duplicates(subset=['topic_url'], keep='last', inplace=True)
    
    new_work = new_work.append(temp, ignore_index = True)
    
print('Количество строк: ', new_work.shape[0])

# удаляю дубликаты
new_work = new_work.drop_duplicates()

print('Количество строк после удаления дублей: ', new_work.shape[0])

  0%|          | 0/96 [00:00<?, ?it/s]

Количество строк:  152812
Количество строк после удаления дублей:  152812


In [56]:
new_work.drop_duplicates(subset=['topic_url'], keep='last', inplace=True)

In [57]:
new_work.shape

(151904, 24)

In [58]:
new_work.sample(2)

,topic_url,category,name,description,price,price_type,item_number,country,region,start_date,...,delivery,delivery_cost,payment,nickname,seller_status,seller_group,rate_pos,rate_neg,user_details,temp_dict
55680,https://forum.ww2.ru/index.php?showtopic=4924290,"ВА, Прочие 1945, Копаные, Архив",трое ножен от итальянского ш/н,"Отправка только по России, кроме Калининграда....","Продано, 2100",блиц цена,none,Россия,Карелия,22 фев 2021,...,По договоренности,none,По договоренности,*******,none,Член клуба форума,58,-0,*******\nЧлен клуба форума\n905 сообщений\nНа ...,"{'Страна продавца': 'Россия', 'Расположение ло..."
2072,https://forum.ww2.ru/index.php?showtopic=4948642,"ВА, Германия, Erkennungsmarke, Архив",Полк связи,На аукцион,Не продано,аукцион,none,Россия,Москва,19 июн 2021,...,По договоренности,300 ₽,По договоренности,*******,none,Член клуба форума,2122,-3,*******\nЧлен клуба форума\n14 132 сообщений\n...,"{'Страна продавца': 'Россия', 'Расположение ло..."


In [47]:
new_work['sale_status'] = new_work.price.copy()

In [48]:
def find_sale_status(work):
    
    if 'Не продано' in work.price or 'Снято с торгов администрацией' in work.price:
            return 0
        
        elif work.price == 'none':
            return 'none'
        
        else:
            return 1  

In [60]:
new_work_actual = new_work[~new_work.category.str.contains('Архив', case=True)]
new_work_archive = new_work[new_work.category.str.contains('Архив', case=True)]

In [61]:
new_work_archive.shape[0] + new_work_actual.shape[0] == new_work.shape[0]

True

In [62]:
new_work.shape

(151904, 24)

In [65]:
new_work_actual['sale_status'] = 2

In [ ]:
new_work_actual['sale_status'] = new_work['sale_status'].str.replace('Снято с торгов администрацией', '0')
new_work['sale_status'] = new_work['sale_status'].str.replace(r'Продано.+', 'Продано', regex=True)

In [46]:
new_work.sale_status.value_counts()

Не продано    60189
Продано       37851
none          24908
1000           3365
500            2443
              ...  
13800             1
45500             1
1718000           1
430               1
1749              1
Name: sale_status, Length: 577, dtype: int64

In [33]:
(new_work.price.value_counts().reset_index()
 .to_excel(file_path + 'sample.xlsx', index=False)
)

In [21]:
new_work['sale_status'] = new_work.apply(find_sale_status, axis=1)

TypeError: argument of type 'float' is not iterable

In [8]:
# убираю Архив

new_work['category'] = new_work['category'].str.replace(', Архив', '')

In [ ]:
#new_work.category.value_counts().reset_index()
#.to_excel(file_path + 'temp.xlsx')

In [ ]:
new_work['region'] = new_work['region'].str.replace('\s+', ' ', regex=True )
#new_work['region'] = new_work['region'].apply(lambda x: x.strip())

new_work['region'] = new_work['region'].str.lower()
new_work['region'] = new_work['region'].str.replace('г. ', '', regex=False)
new_work['region'] = new_work['region'].str.replace('г.', '', regex=False)
new_work['region'] = new_work['region'].str.replace('обл.', 'область', regex=False)

In [ ]:
new_work.name.nunique()

In [ ]:
new_work.region.value_counts().reset_index().to_excel(file_path + 'region.xlsx')